<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import seaborn as sb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import xgboost as xgb


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [3]:
# # Load data
df_train = pd.read_csv('../data/training_set_VU_DM.csv')

#df_test = pd.read_csv('../data/test_set_VU_DM.csv')

#df_example_output = pd.read_csv('../data/submission_sample.csv')

<h1>Data prep<h1\>

In [5]:
def make_score(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    return 0

def date_time(df):
    df_copy = df.copy()
    df_copy['date_time'] = pd.to_datetime(df_copy['date_time'])
    df_copy['hour'] = df_copy['date_time'].dt.hour
    df_copy['day'] = df_copy['date_time'].dt.day
    df_copy['month'] = df_copy['date_time'].dt.month
    return df_copy

In [13]:
df_train['score'] = df_train.apply(lambda row: make_score(row), axis=1)
df_train = date_time(df_train)
df_train = df_train.drop('date_time', axis=1)

<h1>Data split<h1\>

In [15]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 7).split(df_train, groups=df_train['srch_id'])

X_train_inds, X_val_inds = next(gss)

train_data= df_train.iloc[X_train_inds]
X_train = train_data.loc[:, ~train_data.columns.isin(['srch_id','score'])]
y_train = train_data.loc[:, train_data.columns.isin(['score'])]

groups = train_data.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

val_data= df_train.iloc[X_val_inds]

val_groups = test_data.groupby('srch_id').size().to_frame('size')['size'].to_numpy()

#We need to keep the id for later predictions
X_val = val_data.loc[:, ~val_data.columns.isin(['score'])]
y_val = val_data.loc[:, val_data.columns.isin(['score'])]

<h1>Training <h1\>

In [16]:
gbm = lgb.LGBMRanker()

In [1]:
gbm.fit(X_train, y_train, group=groups, eval_set=[(X_val, y_val)], eval_group=[groups, val_groups], early_stopping_rounds=100)

NameError: name 'gbm' is not defined